In [11]:
# Copyright 2022-2023 Sony Semiconductor Solutions Corp. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Get deploy configurations

This notebook explains the workflow for getting deploy configurations from "**Console for AITRIOS**".

Instructions are described in [README.md](./README.md).

## Imports

In [12]:
import numpy as np
import pandas as pd
from console_access_library.client import Client
from IPython.display import display

## Load an instance of "**Console Access Library**" client

Run this cell to restore instance variable 'client_obj' created on a notebook of Set up "**Console Access Library**".

In [13]:
%store -r
client_obj: Client = client_obj

## Get deploy configurations

In [ ]:
# Get an instance of deploy API
deployment_obj = client_obj.get_deployment()

# Call an API to get deploy configurations
try:
    response = deployment_obj.get_deploy_configurations()
except Exception as e:
    # EXCEPTION
    raise e

# response error check
if "result" in response and response["result"] != "SUCCESS":
    # ERROR
    raise ValueError("ERROR", response)

# Create an output table
config_ids = []
model_ids = []
model_version_numbers = []
configurations = response.get("deploy_configurations", [])
for config in configurations:
    model = config.get("model", {})
    model_id = model.get("model_id", "")
    if len(model_id) > 0:
        # Set config ID
        config_id = config.get("config_id", "")
        if len(config_id) > 0:
            config_ids.append(config_id)
        else:
            config_ids.append(np.NaN)
        # Set model ID
        model_ids.append(model_id)
        # Set model version number
        model_version_number = model.get("model_version_number", "")
        if len(model_version_number) > 0:
            model_version_numbers.append(model_version_number)
        else:
            model_version_numbers.append(np.NaN)

if len(config_ids) == 0:
    raise Exception("There is no data in the configuration list.")

output_frame = pd.DataFrame(
    {
        "config_id": config_ids,
        "model_id": model_ids,
        "model_version_number": model_version_numbers
    }
)
output_frame = output_frame.fillna("-")
# setting backup
backup_max_rows = pd.options.display.max_rows
# output limit clear
pd.set_option("display.max_rows", None)
display(output_frame)
# setting restore
pd.set_option("display.max_rows", backup_max_rows)